# Logging the payload with scoring endpoint wrapper

Now, you're ready to create credit risk scoring endpoint wrapper. The goal of the wrapper is to automatically log all input and output payloads from credit risk deployment (created previously).

## Prerequisites to this tutorial:
> * Credit Risk model deployment on Azure ML Service (scoring endpoint)


In this part of the tutorial, you use Azure Machine Learning service to:

> * Set up your testing environment
> * Retrieve the deployment from your workspace
> * Test the deployment locally
> * Deploy the wrapper to ACI
> * Test the wrapper

If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/watson-openscale).

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

**NOTE:** You can also get OpenScale `API_KEY` using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

In [ ]:
CLOUD_API_KEY = '***'

In [ ]:
DB_CREDENTIALS=None
#DB_CREDENTIALS= {"hostname":"","username":"","password":"","database":"","port":"","ssl":True,"sslmode":"","certificate_base64":""}

## Save training data to COS for Openscale reference

In [ ]:
IAM_URL="https://iam.ng.bluemix.net/oidc/token"
COS_API_KEY_ID = "***"
COS_RESOURCE_CRN = "***" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "***" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints
BUCKET_NAME = "***" #example: "credit-risk-training-data"
training_data_file_name="credit_risk_training.csv"

In [ ]:
!rm -rf credit_risk_training.csv
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/credit_risk_training.csv"

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

cos_client = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [ ]:
with open(training_data_file_name, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, training_data_file_name).upload_fileobj(
        Fileobj=file_data
    )

In [ ]:
import pandas as pd
data_df = pd.read_csv("credit_risk_training.csv",
                    dtype={'LoanDuration': int, 'LoanAmount': int, 'InstallmentPercent': int, 'CurrentResidenceDuration': int, 'Age': int, 'ExistingCreditsCount': int, 'Dependents': int})

## Install `ibm-watson-openscale` package

In [ ]:
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade azureml-core --no-cache | tail -n 1

### Action: Restart the kernel

## Set up the workspace

In [ ]:
import os
import urllib.request
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

az_ml_service_credentials = {'tenant_id': '***',
                             'subscription_id': '***', 
                             'resource_group': '***',
                             'workspace_name': '***'}

In [ ]:
interactive_auth = InteractiveLoginAuthentication(tenant_id=az_ml_service_credentials['tenant_id'])

ws = Workspace(subscription_id=az_ml_service_credentials['subscription_id'], 
               resource_group=az_ml_service_credentials['resource_group'], 
               workspace_name=az_ml_service_credentials['workspace_name'], 
               auth=interactive_auth)

In [ ]:
ws.write_config()

In [ ]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

## Set up the environment

Start by setting up a testing environment.

### Import packages

Import the Python packages needed for this tutorial.

In [ ]:
import azureml.core
import os 

print("Azure ML SDK Version: ", azureml.core.VERSION)

### Retrieve deployments

You created a model deployment in your workspace in the previous tutorial. Now, let's list the deployments.

In [ ]:
from azureml.core.webservice import AciWebservice

credit_risk_deployment_name = '***' # service deployment name
credit_risk_scoring_endpoint = None

webservices = AciWebservice.list(ws)
for service in webservices:
    if service.name == credit_risk_deployment_name:
        credit_risk_scoring_endpoint = service.scoring_uri
        
print('scoring endpoint', credit_risk_scoring_endpoint)

## Test endpoint locally

Before creating a scoring endpoint wrapper let's test the original scoring endpoint.

In [ ]:
scoring_data = {"input":[{'CheckingStatus': "0_to_200", 'LoanDuration': 31, 'CreditHistory': "credits_paid_to_date", 'LoanPurpose': "other",'LoanAmount': 1889, 'ExistingSavings': "100_to_500",'EmploymentDuration': "less_1",'InstallmentPercent': 3,'Sex': "female",'OthersOnLoan': "none",'CurrentResidenceDuration': 3, 'OwnsProperty': "savings_insurance",'Age': 32,'InstallmentPlans': "none",'Housing': "own",'ExistingCreditsCount': 1,'Job': "skilled",'Dependents': 1,'Telephone': "none",'ForeignWorker': "yes",
                        },
                        {'CheckingStatus': "no_checking", 'LoanDuration': 13, 'CreditHistory': "credits_paid_to_date", 'LoanPurpose': "car_new",'LoanAmount': 1389, 'ExistingSavings': "100_to_500",'EmploymentDuration': "1_to_4",'InstallmentPercent': 2,'Sex': "male",'OthersOnLoan': "none",'CurrentResidenceDuration': 3, 'OwnsProperty': "savings_insurance",'Age': 25,'InstallmentPlans': "none",'Housing': "own",'ExistingCreditsCount': 2,'Job': "skilled",'Dependents': 2,'Telephone': "none",'ForeignWorker': "yes",
                        }]
              }

In [ ]:
import requests

headers = {'Content-Type':'application/json'}
resp = requests.post(credit_risk_scoring_endpoint, json=scoring_data, headers=headers)
output_data = json.loads(resp.json())

print("POST to url", credit_risk_scoring_endpoint)
print(output_data)

## Add service provider in Openscale

In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *


authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
#authenticator = BearerTokenAuthenticator(bearer_token=IAM_TOKEN) ## uncomment if using IAM token
wos_client = APIClient(authenticator=authenticator)
wos_client.version

In [ ]:
wos_client.data_marts.show()

In [ ]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL, # if using DB2 use DatabaseType.DB2
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=DB_CREDENTIALS['sslmode'],
                        certificate_base64=DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

In [ ]:
AZURE_ENGINE_CREDENTIALS = {
    "client_id": "***",
    "client_secret": "***",
    "subscription_id": "***",
    "tenant": "***"
}

In [ ]:
SERVICE_PROVIDER_NAME = "Azure Machine Learning service"
SERVICE_PROVIDER_DESCRIPTION = "Added by Azure tutorial WOS notebook."

In [ ]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

In [ ]:
service_type = "azure_machine_learning_service"
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type = service_type,
        credentials=AzureCredentials(
            subscription_id= AZURE_ENGINE_CREDENTIALS['subscription_id'], 
            client_id = AZURE_ENGINE_CREDENTIALS['client_id'], 
            client_secret= AZURE_ENGINE_CREDENTIALS['client_secret'],
            tenant = AZURE_ENGINE_CREDENTIALS['tenant']
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id
service_provider_id

In [ ]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id).result
asset_deployment_details

In [ ]:
deployment_id='***' # get the GUID for related deployment
for model_asset_details in asset_deployment_details['resources']:
    if model_asset_details['metadata']['guid']==deployment_id:
        break
model_asset_details

In [ ]:
wos_client.subscriptions.show()

In [ ]:
feature_columns = ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker']
categorical_columns = ['CheckingStatus', 'CreditHistory', 'LoanPurpose', 'ExistingSavings', 'EmploymentDuration', 'Sex', 'OthersOnLoan', 'OwnsProperty', 'InstallmentPlans', 'Housing', 'Job', 'Telephone', 'ForeignWorker']

In [ ]:
azure_asset = Asset(
            asset_id=model_asset_details["entity"]["asset"]["asset_id"],
            name=model_asset_details["entity"]["asset"]["name"],
            url=model_asset_details["entity"]["asset"]["url"],
            asset_type=model_asset_details['entity']['asset']['asset_type'] if 'asset_type' in model_asset_details['entity']['asset'] else 'model',
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        )

In [ ]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointRequest
deployment_scoring_endpoint = model_asset_details['entity']['scoring_endpoint']
scoring_endpoint = ScoringEndpointRequest(url = model_asset_details['entity']['scoring_endpoint']['url'],request_headers = model_asset_details['entity']['scoring_endpoint']['request_headers'],
                                                 credentials = None)  

deployment = AssetDeploymentRequest(
    deployment_id=model_asset_details['metadata']['guid'],
    url=model_asset_details['metadata']['url'],
    name=model_asset_details['entity']['name'],
    description=model_asset_details['entity']['description'],
    deployment_type=model_asset_details['entity']['type'],
    scoring_endpoint = scoring_endpoint
) 

In [ ]:
training_data_reference = TrainingDataReference(type='cos',
                                              location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                        file_name = training_data_file_name),
                                              connection=COSTrainingDataReferenceConnection(
                                                                        resource_instance_id= COS_RESOURCE_CRN,
                                                                        url= COS_ENDPOINT,
                                                                        api_key= COS_API_KEY_ID,
                                                                        iam_url=IAM_URL)
                                               )

In [ ]:
asset_properties = AssetPropertiesRequest(
        label_column="Risk",
        prediction_field='Scored Labels',
        probability_fields=['Scored Probabilities'],
        training_data_reference=training_data_reference,
        training_data_schema=None,
        input_data_schema=None,
        output_data_schema=None,
    )

In [ ]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=azure_asset,
        deployment=deployment,
        asset_properties=asset_properties,
        background_mode=False
).result
subscription_id = subscription_details.metadata.id
subscription_id

In [ ]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

## Implement the wrapper

Once you've tested the deployment, let's implement the wrapper that will:
* call the original scoring endpoint (credit risk)
* conver the scoring request and response to the OpenScale format
* store converted request and response as payload records in OpenScale data mart


To build the correct environment for ACI, provide the following:
* A scoring script to show how to use the scoring wrapper
* An environment file to show what packages need to be installed
* A configuration file to build the ACI


### Create scoring script

Create the scoring script, called score.py, used by the web service call to show how to use the deployment.

You must include two required functions into the scoring script:
* The `init()` function, which typically loads the required credentials into a global object. This function is run only once when the Docker container is started. 

* The `run(input_data)` function uses the original scoring endpoint to predict a value based on the input data. Next it makes format conversions and finally stored payload records in OpenScale data mart.


### ACTION: You need to update below score.py content by changing the following lines:

- `scoring_endpoint` - PUT your scoring endpoint there
- `scoring_headers` - PUT your scoring header there
- `openscale_credentials` - PUT your OpenScale credentials there
- `openscale_subscription_uid` - PUT uid of created subscription there
- you may also need to modify conversion methods to fit your custom format of payloads

### Scoring endpoint wrapper code

In [ ]:
%%writefile score.py
import json
import numpy as np
import os
import time
import requests
import uuid

from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
from ibm_watson_openscale.supporting_classes.enums import *


def convert_user_input_2_openscale(input_data):
    users_records = input_data['input']
    openscale_fields = list(users_records[0])
    openscale_values = [[rec[k] for k in openscale_fields] for rec in users_records] 

    return {'fields':openscale_fields, 'values':openscale_values}


def convert_user_output_2_openscale(output_data):
    output_data = json.loads(output_data)
    users_records = output_data['output']
    openscale_fields = list(users_records[0])
    openscale_values = [[rec[k] for k in openscale_fields] for rec in users_records] 

    return {'fields':openscale_fields, 'values':openscale_values}


def init():
    global openscale_client
    global cloud_api_key
    global openscale_subscription_uid
    global openscale_subscription
    global scoring_endpoint
    global scoring_headers
    global payload_data_set_id
    
    
    scoring_endpoint = '***' #deployment scoring endpoint
    scoring_headers = {'Content-Type': 'application/json'}
    cloud_api_key = '***' # Cloud API key
    openscale_subscription_uid = "***" # subscription id
    authenticator = IAMAuthenticator(apikey=cloud_api_key)
    openscale_client = APIClient(authenticator=authenticator)
    payload_data_set_id = openscale_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=openscale_subscription_uid, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
    
    
def run(input_data):
   
    # ------ CALL SCORING ENDPOINT --------------
    if type(input_data) is str:
        input_data = json.loads(input_data)

    start_time = time.time()        
    response = requests.post(scoring_endpoint, json=input_data, headers=scoring_headers)
    response_time = int((time.time() - start_time)*1000)
    output_data = response.json()

    # ------ PAYLOAD COVERSION TO OPENSCALE FORMAT and LOGGING --------------
    try:
        
        openscale_input = convert_user_input_2_openscale(input_data)
        openscale_output = convert_user_output_2_openscale(output_data)  
        
        records_list=[PayloadRecord(
               request=openscale_input,
               response=openscale_output,
               response_time=response_time
           )]
        openscale_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=records_list)
        
    
    except Exception as e:
        error = str(e)
        print(error)
    finally:
        return output_data
        
        

## Test wrapper locally

In [ ]:
scoring_data

In [ ]:
import score
score.init()

In [ ]:
scores = score.run(scoring_data)

In [ ]:
json.loads(scores)

In [ ]:
wos_client.data_sets.get_records_count(data_set_id=payload_data_set_id)

## Deploy the wrapper as web service

### Create environment file

Next, create an environment file, called myenv.yml, that specifies all of the script's package dependencies. This file is used to ensure that all of those dependencies are installed in the Docker image. This model needs `scikit-learn` and `azureml-sdk`.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 
from azureml.core import Environment

environment = Environment('my-sklearn-environment')

environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'ibm-watson-openscale'
])

### Create configuration file

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. While it depends on your model, the default of 1 core and 1 gigabyte of RAM is usually sufficient for many models. If you feel you need more later, you would have to recreate the image and redeploy the service.

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=0.5, 
                                               memory_gb=0.5, 
                                               tags={"data": "german credit risk",  "method" : "scoring-endpoint-wrapper"}, 
                                               description='Credit risk scoring endpoint wrapper with payload logging')

### Deploy in ACI
Estimated time to complete: **about 7-8 minutes**

Configure the image and deploy. The following code goes through these steps:

1. Build an image using:
   * The scoring file (`score.py`)
   * The environment definition
   * The model file
1. Register that image under the workspace. 
1. Send the image to the ACI container.
1. Start up a container in ACI using the image.
1. Get the web service HTTP endpoint.

## Scoring endpoint creation

In [ ]:
%%time
from azureml.core import Model
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.model import InferenceConfig
from azureml.core.image import ContainerImage


service_name = '***' # wrapper deployment service name

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

inference_config = InferenceConfig(entry_script='score.py', environment=environment)


service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[],
                       inference_config=inference_config,
                       deployment_config=aciconfig,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

## Test deployed wrapper

The following code goes through these steps:
1. Send the scoring request to the web service hosted in ACI. 
2. Print the returned predictions.
3. Check if the scoring request and response has been logged as payload records in OpenScale

In [ ]:
scoring_data = {"input":data_df.sample(50).drop(['Risk'],axis=1).to_dict('records')
              }

In [ ]:
from azureml.core.webservice import AciWebservice

credit_risk_wrapper_deployment_name = '***' # deployed service name
credit_risk_wrapper_endpoint = None

webservices = AciWebservice.list(ws)
for service in webservices:
    if service.name == credit_risk_wrapper_deployment_name:
        credit_risk_wrapper_endpoint = service.scoring_uri
        
print('scoring endpoint', credit_risk_wrapper_endpoint)

In [ ]:
import requests

headers = {'Content-Type':'application/json'}
resp = requests.post(credit_risk_wrapper_endpoint, json=scoring_data, headers=headers)

print("POST to url", credit_risk_wrapper_endpoint)
print(resp.json())

### Check if the payload has been logged

In [ ]:
time.sleep(2)
wos_client.data_sets.get_records_count(data_set_id=payload_data_set_id)

## Quality monitoring and Feedback logging

In [ ]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 10
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

In [ ]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

In [ ]:
!rm additional_feedback_data_v2.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/additional_feedback_data_v2.json

In [ ]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
#print(feedback_dataset)
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

In [ ]:
with open('additional_feedback_data_v2.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)

wos_client.data_sets.store_records(feedback_dataset_id, request_body=additional_feedback_data, background_mode=False)

In [ ]:
# manually records scoring and store as feedback data

# records = [
#     ["no_checking","28","outstanding_credit","appliances","5990","500_to_1000","greater_7","5","male","co-applicant","3","car_other","55","none","free","2","skilled","2","yes","yes","Risk"],
#     ["greater_200","22","all_credits_paid_back","car_used","3376","less_100","less_1","3","female","none","2","car_other","32","none","own","1","skilled","1","none","yes","No Risk"],
#     ["no_checking","39","credits_paid_to_date","vacation","6434","unknown","greater_7","5","male","none","4","car_other","39","none","own","2","skilled","2","yes","yes","Risk"],
#     ["0_to_200","20","credits_paid_to_date","furniture","2442","less_100","unemployed","3","female","none","1","real_estate","42","none","own","1","skilled","1","none","yes","No Risk"],
#     ["greater_200","4","all_credits_paid_back","education","4206","less_100","unemployed","1","female","none","3","savings_insurance","27","none","own","1","management_self-employed","1","none","yes","No Risk"],
#     ["greater_200","23","credits_paid_to_date","car_used","2963","greater_1000","greater_7","4","male","none","4","car_other","46","none","own","2","skilled","1","none","yes","Risk"],
#     ["no_checking","31","prior_payments_delayed","vacation","2673","500_to_1000","1_to_4","3","male","none","2","real_estate","35","stores","rent","1","skilled","2","none","yes","Risk"],
#     ["no_checking","37","prior_payments_delayed","other","6971","500_to_1000","1_to_4","3","male","none","3","savings_insurance","54","none","own","2","skilled","1","yes","yes","Risk"],
#     ["0_to_200","39","prior_payments_delayed","appliances","5685","100_to_500","1_to_4","4","female","none","2","unknown","37","none","own","2","skilled","1","yes","yes","Risk"],
#     ["no_checking","38","prior_payments_delayed","appliances","4990","500_to_1000","greater_7","4","male","none","4","car_other","50","bank","own","2","unemployed","2","yes","yes","Risk"]]

# fields = feature_columns.copy()
# fields.append('Risk')

# payload_scoring =  [{"fields": fields, "values": records}]
# payload_scoring

# wos_client.data_sets.store_records(feedback_dataset_id, request_body=payload_scoring, background_mode=False)

In [ ]:
wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)

In [ ]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result

In [ ]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

## Fairness, Drift and Explainability configuration

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex",
         "majority": ['male'],
         "minority": ['female'],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 50
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id
fairness_monitor_instance_id

In [ ]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False)

### Drift configuration

- Drift requires a trained model to be uploaded manually for Azure. You can train, create and download a drift detection model using template given ( check for Drift detection model generation) [here](https://github.com/IBM-Watson/aios-data-distribution/blob/master/training_statistics_notebook.ipynb)

In [ ]:
!rm -rf creditrisk_azure_drift_detection_model.tar.gz
!wget -O creditrisk_azure_drift_detection_model.tar.gz https://github.com/IBM/watson-openscale-samples/blob/main/IBM%20Cloud/Azure/assets/models/credit_risk/azure_creditrisk_drift_detection_model.tar.gz?raw=true 

In [ ]:
wos_client.monitor_instances.upload_drift_model(
        model_path='creditrisk_azure_drift_detection_model.tar.gz',
        data_mart_id=data_mart_id,
        subscription_id=subscription_id
     )

In [ ]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "min_samples": 50,
    "drift_threshold": 0.1,
    "train_drift_model": False,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id

In [ ]:
drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False)

In [ ]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

### Explainability configuration

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

In [ ]:
explainability_monitor_id = explainability_details.metadata.id
explainability_monitor_id

In [ ]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types).result
print(result)
explanation_task_id=result.to_dict()['metadata']['explanation_task_ids'][0]
explanation_task_id

In [ ]:
wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=explanation_task_id).result.to_dict()

## Congratulations

You have finished the tutorial for IBM Watson OpenScale and Azure Machine Learning Studio. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit Azure model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.